## Лабораторная работа №1 <br>

### Цель работы: <br>

- изучить операции загрузки и выгрузки данных в HDFS, <br>
- ознакомиться с базовыми операциями Apache Spark в spark-shell, <br>
- создать проект по обработке данных в IDE, <br>
- отладить анализ данных велопарковок на локальном компьютере, <br>
- запустить анализ данных велопарковок на сервере. <br>

Задача: <br>
Для данных велопарковок Сан-Франциско:<br>
 1. Найти велосипед с максимальным временем пробега.<br>
 2. Найти наибольшее геодезическое расстояние между станциями.<br>
 3. Найти путь велосипеда с максимальным временем пробега через станции.<br>
 4. Найти количество велосипедов в системе.<br>
 5. Найти пользователей потративших на поездки более 3 часов.<br>

In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark import SparkContext, SparkConf

from typing import NamedTuple
from datetime import datetime
import numpy as np
conf = SparkConf().setAppName("Lab1_Script").setMaster('yarn')

In [4]:
sc = SparkContext(conf=conf)

In [5]:
from pyspark.sql import SparkSession 
spark = SparkSession(sc) 
trip_df = spark.read.csv("trips.csv") 

In [6]:
tripData = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv("trips.csv")

tripData

DataFrame[id: int, duration: int, start_date: timestamp, start_station_name: string, start_station_id: int, end_date: timestamp, end_station_name: string, end_station_id: int, bike_id: int, subscription_type: string, zip_code: string]

In [7]:
tripData.printSchema()

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: timestamp (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [8]:
tripData.show(n=10)

+----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|         start_date|  start_station_name|start_station_id|           end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|4576|      63|               null|South Van Ness at...|              66|2013-08-29 14:14:00|South Van Ness at...|            66|    520|       Subscriber|   94127|
|4607|    null|2013-08-29 14:42:00|  San Jose City Hall|              10|2013-08-29 14:43:00|  San Jose City Hall|            10|    661|       Subscriber|   95138|
|4130|      71|2013-08-29 10:16:00|Mountain View Cit...|              27|2013-08-29 10:17:00|Mountain View Cit...|            27|     48|       Subscriber|   97214|
|4251|    

In [9]:
? tripData.dropna

Signature:  tripData.dropna(how='any', thresh=None, subset=None)
Docstring:
Returns a new :class:`DataFrame` omitting rows with null values.
:func:`DataFrame.dropna` and :func:`DataFrameNaFunctions.drop` are aliases of each other.

.. versionadded:: 1.3.1

Parameters
----------
how : str, optional
    'any' or 'all'.
    If 'any', drop a row if it contains any nulls.
    If 'all', drop a row only if all its values are null.
thresh: int, optional
    default None
    If specified, drop rows that have less than `thresh` non-null values.
    This overwrites the `how` parameter.
subset : str, tuple or list, optional
    optional list of column names to consider.

Examples
--------
>>> df4.na.drop().show()
+---+------+-----+
|age|height| name|
+---+------+-----+
| 10|    80|Alice|
+---+------+-----+
File:      ~/.local/lib/python3.9/site-packages/pyspark/sql/dataframe.py
Type:      method


In [10]:
tripData.dropna().show(n=10)

+----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|         start_date|  start_station_name|start_station_id|           end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|4130|      71|2013-08-29 10:16:00|Mountain View Cit...|              27|2013-08-29 10:17:00|Mountain View Cit...|            27|     48|       Subscriber|   97214|
|4251|      77|2013-08-29 11:29:00|  San Jose City Hall|              10|2013-08-29 11:30:00|  San Jose City Hall|            10|     26|       Subscriber|   95060|
|4299|      83|2013-08-29 12:02:00|South Van Ness at...|              66|2013-08-29 12:04:00|      Market at 10th|            67|    319|       Subscriber|   94103|
|4927|    

In [11]:
tripData.describe().show()

+-------+------------------+------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+-------------------+
|summary|                id|          duration|  start_station_name|  start_station_id|    end_station_name|    end_station_id|           bike_id|subscription_type|           zip_code|
+-------+------------------+------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+-------------------+
|  count|            985352|            985351|              985352|            985352|              985352|            985352|            985352|           985352|             978484|
|   mean|521401.35102481145|1092.1337716204682|                null|58.060558054380564|                null|58.043319544690625|426.07050779822845|             null|  2008421.845039644|
| stddev| 245889.2855318234| 25689.80328664839|                null|16.9989

In [12]:
stationData = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y')\
.csv("stations.csv")

stationData.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: timestamp (nullable = true)



In [13]:
stationData.show(n=5)

+---+--------------------+------------------+-------------------+----------+--------+-------------------+
| id|                name|               lat|               long|dock_count|    city|  installation_date|
+---+--------------------+------------------+-------------------+----------+--------+-------------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|        27|San Jose|2013-08-06 00:00:00|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|        15|San Jose|2013-08-05 00:00:00|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|        11|San Jose|2013-08-06 00:00:00|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|        19|San Jose|2013-08-05 00:00:00|
|  6|    San Pedro Square|37.336721000000004|        -121.894074|        15|San Jose|2013-08-07 00:00:00|
+---+--------------------+------------------+-------------------+----------+--------+-------------------+
only showing top 5 rows



In [14]:
stationData.describe().show()

+-------+------------------+--------------------+-------------------+-------------------+-----------------+-------------+
|summary|                id|                name|                lat|               long|       dock_count|         city|
+-------+------------------+--------------------+-------------------+-------------------+-----------------+-------------+
|  count|                70|                  70|                 70|                 70|               70|           70|
|   mean|              43.0|                null|  37.59024338428572|-122.21841616428571|17.65714285714286|         null|
| stddev|24.166091947189145|                null|0.20347253639672502|0.20944604979644524|4.010441857493954|         null|
|    min|                 2|       2nd at Folsom|          37.329732|        -122.418954|               11|Mountain View|
|    max|                84|Yerba Buena Cente...|           37.80477|        -121.877349|               27|     San Jose|
+-------+---------------

In [15]:
tripData.printSchema()
stationData.printSchema()

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: timestamp (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: timestamp (nullable = true)



In [16]:
stationsView = stationData.select(stationData['id'], stationData['name'], stationData['lat'], stationData['long'])
stationsView.show()

+---+--------------------+------------------+-------------------+
| id|                name|               lat|               long|
+---+--------------------+------------------+-------------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|
|  6|    San Pedro Square|37.336721000000004|        -121.894074|
|  7|Paseo de San Antonio|         37.333798|-121.88694299999999|
|  8| San Salvador at 1st|         37.330165|-121.88583100000001|
|  9|           Japantown|         37.348742|-121.89471499999999|
| 10|  San Jose City Hall|         37.337391|        -121.886995|
| 11|         MLK Library|         37.335885|-121.88566000000002|
| 12|SJSU 4th at San C...|         37.332808|-121.88389099999999|
| 13|       St James Park|         37.339301|-121.88993700000002|
| 14|Arena

In [17]:
startTrips = tripData.select(tripData.id, tripData.duration, tripData.start_station_id).withColumnRenamed('id', 'trip_id').join(stationsView, tripData.start_station_id == stationsView.id)
startTrips = startTrips.drop('id')

In [18]:
startTrips.show()

+-------+--------+----------------+--------------------+------------------+-------------------+
|trip_id|duration|start_station_id|                name|               lat|               long|
+-------+--------+----------------+--------------------+------------------+-------------------+
|   4576|      63|              66|South Van Ness at...|         37.774814|        -122.418954|
|   4607|    null|              10|  San Jose City Hall|         37.337391|        -121.886995|
|   4130|      71|              27|Mountain View Cit...|         37.389218|        -122.081896|
|   4251|      77|              10|  San Jose City Hall|         37.337391|        -121.886995|
|   4299|      83|              66|South Van Ness at...|         37.774814|        -122.418954|
|   4927|     103|              59| Golden Gate at Polk|         37.781332|        -122.418603|
|   4500|     109|               4|Santa Clara at Al...|         37.333988|        -121.894902|
|   4563|     111|               8| San 

In [19]:
tripData.printSchema()
stationData.printSchema()

root
 |-- id: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- start_date: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_date: timestamp (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- subscription_type: string (nullable = true)
 |-- zip_code: string (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- dock_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- installation_date: timestamp (nullable = true)



In [20]:
stationData.createOrReplaceTempView("stations")
tripData.createOrReplaceTempView("trips")

In [21]:
endTrips = spark.sql("""
SELECT trips.id as trip_id, trips.end_station_id, trips.duration, stations.name as station_name, stations.lat, stations.long 
FROM trips INNER JOIN stations 
    ON trips.end_station_id==stations.id
""")

In [22]:
endTrips.show(10)

+-------+--------------+--------+--------------------+------------------+-------------------+
|trip_id|end_station_id|duration|        station_name|               lat|               long|
+-------+--------------+--------+--------------------+------------------+-------------------+
|   4576|            66|      63|South Van Ness at...|         37.774814|        -122.418954|
|   4607|            10|    null|  San Jose City Hall|         37.337391|        -121.886995|
|   4130|            27|      71|Mountain View Cit...|         37.389218|        -122.081896|
|   4251|            10|      77|  San Jose City Hall|         37.337391|        -121.886995|
|   4299|            67|      83|      Market at 10th|37.776619000000004|-122.41738500000001|
|   4927|            59|     103| Golden Gate at Polk|         37.781332|        -122.418603|
|   4500|             5|     109|    Adobe on Almaden|         37.331415|          -121.8932|
|   4563|             8|     111| San Salvador at 1st|      

In [23]:
test1 = spark.sql("""
SELECT start_station_name, avg(duration)
FROM trips
GROUP BY trips.start_station_name
ORDER BY avg(duration) DESC
""")

In [24]:
test1_max_val = test1.agg({"avg(duration)": "max"}).collect()[0]

In [25]:
test1_max_val

Row(max(avg(duration))=7230.231034482758)

In [26]:
spark.sql("""
SELECT start_station_name, avg(duration)
FROM trips
GROUP BY trips.start_station_name
ORDER BY avg(duration) DESC
""").show()

+--------------------+------------------+
|  start_station_name|     avg(duration)|
+--------------------+------------------+
|University and Em...| 7230.231034482758|
|Redwood City Medi...|  4764.68287037037|
|San Jose Civic Ce...| 4720.621422376409|
|       Park at Olive| 4686.397612488521|
|California Ave Ca...| 4502.620639534884|
|Redwood City Publ...|3697.0892307692307|
|Rengstorff Avenue...| 3544.797270955166|
|Palo Alto Caltrai...| 3158.331498866947|
|San Mateo County ...|3002.0827067669175|
|South Van Ness at...|2936.8873503613395|
|San Antonio Shopp...| 2508.434736091298|
|Cowper at University| 2493.220572640509|
|    Broadway at Main|2481.2537313432836|
|Redwood City Calt...|  2405.29409190372|
|           Japantown|2297.0913838120105|
|San Antonio Caltr...|2103.7238932071646|
|SJSU 4th at San C...| 1995.366021236727|
|Washington at Kea...|1979.3077445652175|
|          Mezes Park|1918.1354359925788|
|Arena Green / SAP...|1888.3390476190475|
+--------------------+------------

In [27]:
spark.sql("""
SELECT trips.bike_id, trips.start_date, trips.end_date, stations.name
FROM trips INNER JOIN stations
    ON trips.start_station_id == stations.id
WHERE 
    bike_id == 523 
    AND start_date > make_timestamp(2013, 12, 25, 0, 0, 0) 
    AND start_date <  make_timestamp(2013, 12, 26, 0, 0, 0)
""").show()

+-------+-------------------+-------------------+--------------------+
|bike_id|         start_date|           end_date|                name|
+-------+-------------------+-------------------+--------------------+
|    523|2013-12-25 01:51:00|2013-12-25 02:12:00|       5th at Howard|
|    523|2013-12-25 02:14:00|2013-12-25 02:40:00|Mechanics Plaza (...|
|    523|2013-12-25 02:50:00|2013-12-25 03:22:00|Embarcadero at Va...|
|    523|2013-12-25 03:32:00|2013-12-25 03:52:00|Embarcadero at Sa...|
+-------+-------------------+-------------------+--------------------+



### Task: `Найти велосипед с максимальным временем пробега.`

In [28]:
df_1 = spark.sql("""
SELECT bike_id, duration, start_date, end_date 
FROM trips 
WHERE 
    start_date > make_timestamp(2013, 12, 25, 0, 0, 0) 
    AND start_date <  make_timestamp(2013, 12, 26, 0, 0, 0)
""")
df_1.show(10)

+-------+--------+-------------------+-------------------+
|bike_id|duration|         start_date|           end_date|
+-------+--------+-------------------+-------------------+
|    544|     497|2013-12-25 01:41:00|2013-12-25 01:49:00|
|    470|    1577|2013-12-25 01:46:00|2013-12-25 02:12:00|
|    494|    1409|2013-12-25 01:49:00|2013-12-25 02:12:00|
|    523|    1241|2013-12-25 01:51:00|2013-12-25 02:12:00|
|    578|    1230|2013-12-25 01:52:00|2013-12-25 02:12:00|
|    322|    1134|2013-12-25 01:53:00|2013-12-25 02:12:00|
|    480|     888|2013-12-25 01:57:00|2013-12-25 02:12:00|
|    470|    1662|2013-12-25 02:13:00|2013-12-25 02:41:00|
|    578|    1559|2013-12-25 02:14:00|2013-12-25 02:40:00|
|    523|    1560|2013-12-25 02:14:00|2013-12-25 02:40:00|
+-------+--------+-------------------+-------------------+
only showing top 10 rows



In [29]:
from pyspark.sql import functions as F

In [30]:
dfs_time_all = df_1.withColumn('date_travel_hour_minute', F.hour(df_1.end_date)*60 - F.hour(df_1.start_date)*60 + F.minute(df_1.end_date) - F.minute(df_1.start_date))

In [31]:
dfs_time_all.show(10)

+-------+--------+-------------------+-------------------+-----------------------+
|bike_id|duration|         start_date|           end_date|date_travel_hour_minute|
+-------+--------+-------------------+-------------------+-----------------------+
|    544|     497|2013-12-25 01:41:00|2013-12-25 01:49:00|                      8|
|    470|    1577|2013-12-25 01:46:00|2013-12-25 02:12:00|                     26|
|    494|    1409|2013-12-25 01:49:00|2013-12-25 02:12:00|                     23|
|    523|    1241|2013-12-25 01:51:00|2013-12-25 02:12:00|                     21|
|    578|    1230|2013-12-25 01:52:00|2013-12-25 02:12:00|                     20|
|    322|    1134|2013-12-25 01:53:00|2013-12-25 02:12:00|                     19|
|    480|     888|2013-12-25 01:57:00|2013-12-25 02:12:00|                     15|
|    470|    1662|2013-12-25 02:13:00|2013-12-25 02:41:00|                     28|
|    578|    1559|2013-12-25 02:14:00|2013-12-25 02:40:00|                     26|
|   

In [32]:
# Группируем по айди байка и смотрим продолжительность поездок
i = dfs_time_all.groupBy('bike_id').agg(F.max('date_travel_hour_minute'))
i.show()
# Итого саммая продолжительная поездка
j = dfs_time_all.agg(F.max('date_travel_hour_minute'))
j.show()

+-------+----------------------------+
|bike_id|max(date_travel_hour_minute)|
+-------+----------------------------+
|    463|                          19|
|    392|                         134|
|    540|                          24|
|    516|                         155|
|     85|                          12|
|    458|                          11|
|    322|                          31|
|    513|                         212|
|    362|                         276|
|    613|                         124|
|    375|                          34|
|    368|                         140|
|    596|                          24|
|    497|                           4|
|    412|                         197|
|    300|                           5|
|    626|                           4|
|    577|                          89|
|    271|                          23|
|    329|                           7|
+-------+----------------------------+
only showing top 20 rows

+----------------------------+
|max(da

In [33]:
# Сам велосипед где поездка равна самой большой (см. выше) 

k = dfs_time_all[dfs_time_all.date_travel_hour_minute == 399] 
k.show()

+-------+--------+-------------------+-------------------+-----------------------+
|bike_id|duration|         start_date|           end_date|date_travel_hour_minute|
+-------+--------+-------------------+-------------------+-----------------------+
|    555|   23909|2013-12-25 14:52:00|2013-12-25 21:31:00|                    399|
+-------+--------+-------------------+-------------------+-----------------------+



### Task: `Найти наибольшее геодезическое расстояние между станциями.`

In [34]:
stations = spark.read.option("header", True).csv("stations.csv")

In [35]:
stations.show()

+---+--------------------+------------------+-------------------+----------+------------+-----------------+
| id|                name|               lat|               long|dock_count|        city|installation_date|
+---+--------------------+------------------+-------------------+----------+------------+-----------------+
|  2|San Jose Diridon ...|         37.329732|-121.90178200000001|        27|    San Jose|         8/6/2013|
|  3|San Jose Civic Ce...|         37.330698|        -121.888979|        15|    San Jose|         8/5/2013|
|  4|Santa Clara at Al...|         37.333988|        -121.894902|        11|    San Jose|         8/6/2013|
|  5|    Adobe on Almaden|         37.331415|          -121.8932|        19|    San Jose|         8/5/2013|
|  6|    San Pedro Square|37.336721000000004|        -121.894074|        15|    San Jose|         8/7/2013|
|  7|Paseo de San Antonio|         37.333798|-121.88694299999999|        15|    San Jose|         8/7/2013|
|  8| San Salvador at 1st|  

In [36]:
all_combinations = stations.selectExpr('name as A', 'lat as A_lat', 'long as A_long').join(stations.selectExpr('name as B', 'lat as B_lat', 'long as B_long'))

In [37]:
all_combinations.show()

+--------------------+---------+-------------------+--------------------+------------------+-------------------+
|                   A|    A_lat|             A_long|                   B|             B_lat|             B_long|
+--------------------+---------+-------------------+--------------------+------------------+-------------------+
|San Jose Diridon ...|37.329732|-121.90178200000001|San Jose Diridon ...|         37.329732|-121.90178200000001|
|San Jose Diridon ...|37.329732|-121.90178200000001|San Jose Civic Ce...|         37.330698|        -121.888979|
|San Jose Diridon ...|37.329732|-121.90178200000001|Santa Clara at Al...|         37.333988|        -121.894902|
|San Jose Diridon ...|37.329732|-121.90178200000001|    Adobe on Almaden|         37.331415|          -121.8932|
|San Jose Diridon ...|37.329732|-121.90178200000001|    San Pedro Square|37.336721000000004|        -121.894074|
|San Jose Diridon ...|37.329732|-121.90178200000001|Paseo de San Antonio|         37.333798|-121

In [38]:
all_different_combinations = all_combinations[all_combinations.A != all_combinations.B]

In [39]:
from math import sqrt
def euclidian_distance(ax, ay, bx, by):
    return sqrt((ax-bx)**2 + (ay-by)**2)

In [40]:
distances_between_stations = all_different_combinations.rdd.map(lambda row: (row.A, row.B, euclidian_distance(float(row.A_lat), float(row.A_long), float(row.B_lat), float(row.B_long)) ))

In [41]:
distances_between_stations.max(lambda row: row[2])

('SJSU - San Salvador at 9th', 'Embarcadero at Sansome', 0.7058482821754397)

### Task: `Найти путь велосипеда с максимальным временем пробега через станции.`

In [42]:
df_n3 = spark.sql("""
SELECT bike_id, duration, start_date, end_date 
FROM trips 
WHERE 
    start_date > make_timestamp(2013, 12, 25, 0, 0, 0) 
    AND start_date <  make_timestamp(2013, 12, 26, 0, 0, 0)
""")
df_n3.show(20)

+-------+--------+-------------------+-------------------+
|bike_id|duration|         start_date|           end_date|
+-------+--------+-------------------+-------------------+
|    544|     497|2013-12-25 01:41:00|2013-12-25 01:49:00|
|    470|    1577|2013-12-25 01:46:00|2013-12-25 02:12:00|
|    494|    1409|2013-12-25 01:49:00|2013-12-25 02:12:00|
|    523|    1241|2013-12-25 01:51:00|2013-12-25 02:12:00|
|    578|    1230|2013-12-25 01:52:00|2013-12-25 02:12:00|
|    322|    1134|2013-12-25 01:53:00|2013-12-25 02:12:00|
|    480|     888|2013-12-25 01:57:00|2013-12-25 02:12:00|
|    470|    1662|2013-12-25 02:13:00|2013-12-25 02:41:00|
|    578|    1559|2013-12-25 02:14:00|2013-12-25 02:40:00|
|    523|    1560|2013-12-25 02:14:00|2013-12-25 02:40:00|
|    322|    1518|2013-12-25 02:14:00|2013-12-25 02:40:00|
|    494|    1470|2013-12-25 02:15:00|2013-12-25 02:40:00|
|    482|    2430|2013-12-25 02:15:00|2013-12-25 02:56:00|
|    634|    1434|2013-12-25 02:16:00|2013-12-25 02:40:0

In [43]:
trip_max_vremya = df_n3.agg({"duration": "max"}).collect()[0]
print(trip_max_vremya)

Row(max(duration)=91897)


In [46]:
# Выводим строку для данного велосипеда

df_n3[df_n3.duration == 91897].show()

+-------+--------+-------------------+-------------------+
|bike_id|duration|         start_date|           end_date|
+-------+--------+-------------------+-------------------+
|    656|   91897|2013-12-25 07:58:00|2013-12-26 09:30:00|
+-------+--------+-------------------+-------------------+



In [47]:
STATION_BIKE = tripData[tripData.bike_id == 656]
STATION_BIKE.show(10)

+-----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|   id|duration|         start_date|  start_station_name|start_station_id|           end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+-----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|15833|    3297|2013-09-09 17:31:00|SJSU 4th at San C...|              12|2013-09-09 18:26:00|SJSU 4th at San C...|            12|    656|         Customer|   93035|
|17053|     837|2013-09-10 18:09:00|SJSU 4th at San C...|              12|2013-09-10 18:23:00|San Jose Diridon ...|             2|    656|       Subscriber|   95110|
|17084|    1093|2013-09-10 18:26:00|San Jose Diridon ...|               2|2013-09-10 18:44:00|SJSU - San Salvad...|            16|    656|       Subscriber|   95110|
|196

In [49]:
#Информация о проделанном пути


STATION_BIKE = STATION_BIKE.select(STATION_BIKE['start_station_name'], STATION_BIKE['end_station_name'])
STATION_BIKE.show(20) 

+--------------------+--------------------+
|  start_station_name|    end_station_name|
+--------------------+--------------------+
|SJSU 4th at San C...|SJSU 4th at San C...|
|SJSU 4th at San C...|San Jose Diridon ...|
|San Jose Diridon ...|SJSU - San Salvad...|
|Santa Clara at Al...|Paseo de San Antonio|
|Paseo de San Antonio|SJSU 4th at San C...|
|SJSU 4th at San C...|Paseo de San Antonio|
|Paseo de San Antonio|SJSU 4th at San C...|
|SJSU 4th at San C...|SJSU - San Salvad...|
|Santa Clara at Al...|San Jose Diridon ...|
|San Jose Diridon ...|  San Jose City Hall|
|  San Jose City Hall| San Salvador at 1st|
| San Salvador at 1st|San Jose Diridon ...|
|San Jose Diridon ...|    San Pedro Square|
|San Jose Diridon ...|Santa Clara at Al...|
|Santa Clara at Al...|San Jose Civic Ce...|
|San Jose Civic Ce...|San Jose Civic Ce...|
|San Jose Civic Ce...|San Jose Diridon ...|
|San Jose Diridon ...|    Adobe on Almaden|
|  San Jose City Hall| San Salvador at 1st|
| San Salvador at 1st|         M

In [54]:

df_data = df_1.withColumn(
    'diff',
    (F.datediff(F.to_date(df_1.end_date), F.to_date(df_1.start_date))*24*60*60+
    (F.unix_timestamp(F.to_timestamp(F.col('end_date')))) - 
    (F.unix_timestamp(F.to_timestamp(F.col('start_date')))))/60
)

In [55]:
df_data.dropna().show(n=10)

+-------+--------+-------------------+-------------------+----+
|bike_id|duration|         start_date|           end_date|diff|
+-------+--------+-------------------+-------------------+----+
|    544|     497|2013-12-25 01:41:00|2013-12-25 01:49:00| 8.0|
|    470|    1577|2013-12-25 01:46:00|2013-12-25 02:12:00|26.0|
|    494|    1409|2013-12-25 01:49:00|2013-12-25 02:12:00|23.0|
|    523|    1241|2013-12-25 01:51:00|2013-12-25 02:12:00|21.0|
|    578|    1230|2013-12-25 01:52:00|2013-12-25 02:12:00|20.0|
|    322|    1134|2013-12-25 01:53:00|2013-12-25 02:12:00|19.0|
|    480|     888|2013-12-25 01:57:00|2013-12-25 02:12:00|15.0|
|    470|    1662|2013-12-25 02:13:00|2013-12-25 02:41:00|28.0|
|    578|    1559|2013-12-25 02:14:00|2013-12-25 02:40:00|26.0|
|    523|    1560|2013-12-25 02:14:00|2013-12-25 02:40:00|26.0|
+-------+--------+-------------------+-------------------+----+
only showing top 10 rows



In [56]:
trip_max_vremya = df_data.agg({"diff": "max"}).collect()[0]
print(trip_max_vremya)

Row(max(diff)=2972.0)


In [57]:
k = df_data[df_data.diff == 2972] #Выводим велосипед с самой долгой поездкой (diff) Смотрим его duration
k.show()

+-------+--------+-------------------+-------------------+------+
|bike_id|duration|         start_date|           end_date|  diff|
+-------+--------+-------------------+-------------------+------+
|    656|   91897|2013-12-25 07:58:00|2013-12-26 09:30:00|2972.0|
+-------+--------+-------------------+-------------------+------+



### Task: `Найти количество велосипедов в системе.`

In [58]:
from pyspark.sql.functions import col, countDistinct

In [59]:
tripData.show(5)
 
    
df_id = spark.sql("""
SELECT bike_id 
FROM trips 
""")
df_id.show(10)   

+----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|  id|duration|         start_date|  start_station_name|start_station_id|           end_date|    end_station_name|end_station_id|bike_id|subscription_type|zip_code|
+----+--------+-------------------+--------------------+----------------+-------------------+--------------------+--------------+-------+-----------------+--------+
|4576|      63|               null|South Van Ness at...|              66|2013-08-29 14:14:00|South Van Ness at...|            66|    520|       Subscriber|   94127|
|4607|    null|2013-08-29 14:42:00|  San Jose City Hall|              10|2013-08-29 14:43:00|  San Jose City Hall|            10|    661|       Subscriber|   95138|
|4130|      71|2013-08-29 10:16:00|Mountain View Cit...|              27|2013-08-29 10:17:00|Mountain View Cit...|            27|     48|       Subscriber|   97214|
|4251|    

In [60]:
print('Кол-во велосипедов: ', df_id.distinct().count())

Кол-во велосипедов:  700


### Task: `5 Найти пользователей потративших на поездки более 3 часов.` 

In [65]:
# Вывели структуру с датами

df_1.show(10) 

+-------+--------+-------------------+-------------------+
|bike_id|duration|         start_date|           end_date|
+-------+--------+-------------------+-------------------+
|    544|     497|2013-12-25 01:41:00|2013-12-25 01:49:00|
|    470|    1577|2013-12-25 01:46:00|2013-12-25 02:12:00|
|    494|    1409|2013-12-25 01:49:00|2013-12-25 02:12:00|
|    523|    1241|2013-12-25 01:51:00|2013-12-25 02:12:00|
|    578|    1230|2013-12-25 01:52:00|2013-12-25 02:12:00|
|    322|    1134|2013-12-25 01:53:00|2013-12-25 02:12:00|
|    480|     888|2013-12-25 01:57:00|2013-12-25 02:12:00|
|    470|    1662|2013-12-25 02:13:00|2013-12-25 02:41:00|
|    578|    1559|2013-12-25 02:14:00|2013-12-25 02:40:00|
|    523|    1560|2013-12-25 02:14:00|2013-12-25 02:40:00|
+-------+--------+-------------------+-------------------+
only showing top 10 rows



In [66]:
# Вычли разницу во времени

df_data = df_1.withColumn(
    'diff',
    (F.datediff(F.to_date(df_1.end_date), F.to_date(df_1.start_date))*24*60*60+
    (F.unix_timestamp(F.to_timestamp(F.col('end_date')))) - 
    (F.unix_timestamp(F.to_timestamp(F.col('start_date')))))/60
)
k = df_data[df_data.diff > 3*60] 
k.show(100) 

+-------+--------+-------------------+-------------------+------+
|bike_id|duration|         start_date|           end_date|  diff|
+-------+--------+-------------------+-------------------+------+
|    656|   91897|2013-12-25 07:58:00|2013-12-26 09:30:00|2972.0|
|    366|   16375|2013-12-25 09:36:00|2013-12-25 14:09:00| 273.0|
|    371|   16368|2013-12-25 09:36:00|2013-12-25 14:09:00| 273.0|
|    127|   11192|2013-12-25 09:40:00|2013-12-25 12:46:00| 186.0|
|    643|   10979|2013-12-25 09:43:00|2013-12-25 12:46:00| 183.0|
|    412|   11823|2013-12-25 11:14:00|2013-12-25 14:31:00| 197.0|
|    468|   16841|2013-12-25 12:45:00|2013-12-25 17:26:00| 281.0|
|    362|   16598|2013-12-25 12:49:00|2013-12-25 17:25:00| 276.0|
|    562|   16415|2013-12-25 12:52:00|2013-12-25 17:25:00| 273.0|
|    414|   11308|2013-12-25 12:52:00|2013-12-25 16:00:00| 188.0|
|    578|   12311|2013-12-25 12:56:00|2013-12-25 16:21:00| 205.0|
|    520|   15928|2013-12-25 13:00:00|2013-12-25 17:25:00| 265.0|
|    627| 